In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F

In [0]:
import os
os.listdir("drive/datasets")

['flower_data',
 'anna.txt',
 'cat_to_name.json',
 'checkpoint.pth',
 'checkpoint_classifier.pth',
 'intermediate_weights.csv',
 'code_corpus.csv',
 'dictionary.csv',
 'dictionary_large.csv',
 'code_corpus_large.csv',
 'SCRNN.net',
 'smells-dataset',
 'features_code2vec.csv',
 'features_lstm.csv',
 'smells_train.csv',
 'smells_valid.csv',
 'smells_test.csv']

In [0]:
import pandas as pd
import numpy as np

data = pd.read_csv("drive/datasets/features_lstm.csv")
#data_code2vec = pd.read_csv("drive/datasets/features_code2vec.csv")

data = data.iloc[:,1:]
#data_code2vec = data_code2vec.iloc[:,1:]

#data_common = pd.merge(data, data_code2vec, on='id')

# print(len(data))
# print(len(data_code2vec))
# print(len(data_common))

# data_common = data_common.drop(columns=['sevrity', 'smell_x'])

# data_common = data_common.rename(columns={'smell_y': 'smell'})

# data = data_common

1379
1183
1183


In [0]:
print(len(data[data['smell'] == 'data-class']))
print(len(data[data['smell'] == 'god-class']))
print(len(data[data['smell'] == 'feature-envy']))
print(len(data[data['smell'] == 'long-method']))


data = data.rename(columns={'sevrity': 'severity'})
data.head()

367
364
226
226


,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,376_y,377_y,378_y,379_y,380_y,381_y,382_y,383_y,severity,smell
0,-0.113796,0.084324,-0.114097,-0.224774,-0.227246,0.118165,-0.112589,0.004592,0.195760,-0.056564,...,-0.570086,-0.229752,-0.044707,-0.540615,-0.644185,-0.097805,-0.060193,0.518125,3,data-class
1,-0.073298,0.098071,-0.228914,-0.185084,-0.119284,0.118947,-0.159412,-0.061848,0.141519,-0.068608,...,0.156044,0.050116,0.240378,-0.174545,-0.149885,0.154067,0.167369,-0.237400,4,god-class
2,-0.111986,0.090784,-0.107854,-0.236817,-0.109894,0.079085,-0.100038,0.057145,0.257870,-0.095232,...,0.142108,0.165253,-0.134043,0.185795,-0.450761,-0.232658,-0.171978,0.189045,3,data-class
3,-0.079752,0.067778,-0.251782,-0.049088,-0.117858,0.122254,-0.108178,-0.040676,0.266969,-0.079081,...,0.341786,0.162538,-0.043623,-0.226228,0.564148,-0.110596,-0.019506,0.074474,2,feature-envy
4,-0.106490,0.072026,-0.075713,-0.253144,-0.117503,0.070339,-0.075484,0.055864,0.229228,-0.070227,...,0.365456,-0.077600,0.412811,-0.009280,0.010046,-0.185803,0.011436,0.259090,3,god-class


In [0]:
data = data[data['smell'] == 'feature-envy']

data['severity'] = data.apply(lambda x: 1 if x['severity'] in [3,4] else 0, axis = 1)
data = data.rename(columns={'severity':'label'})

In [0]:
data = data.drop(columns=['smell'])

In [0]:
rndperm = np.random.permutation(data.shape[0])

data = data.iloc[rndperm[:],:].copy()

In [0]:
data.head()

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,375_y,376_y,377_y,378_y,379_y,380_y,381_y,382_y,383_y,label
1163,-0.145700,0.118192,-0.031552,-0.248118,-0.064677,0.132315,-0.069630,0.012900,0.125375,-0.117974,...,0.197851,-0.725822,-0.744570,0.994790,-0.775242,0.688465,-0.860790,-0.516777,-0.348757,0
617,-0.101418,0.138110,-0.141301,-0.194752,-0.071304,0.109208,-0.186825,-0.023373,0.139718,-0.096214,...,0.335458,-0.069941,0.011133,-0.359929,0.138720,0.794518,-0.526687,-0.241594,-0.240812,0
791,-0.072871,0.051296,-0.248364,-0.073282,-0.046383,0.075795,-0.076810,-0.031859,0.140451,-0.101411,...,0.690238,-0.032537,-0.627023,0.362687,-0.593341,0.589254,-0.443808,0.294625,-0.553835,0
549,-0.106933,0.077259,-0.221718,-0.289869,-0.117346,0.135075,-0.151017,-0.023453,0.240949,-0.075439,...,-0.561210,0.135872,0.313277,-0.106050,0.206891,0.415308,-0.007933,0.255159,-0.249885,0
874,-0.046034,0.030986,-0.254069,-0.274502,-0.092662,0.114870,-0.110452,0.015921,0.154487,-0.086019,...,-0.036260,0.384238,-0.257764,-0.745335,0.642678,0.227680,-0.324755,-0.474306,0.124328,0


In [0]:
X = data.loc[:,:'label']
y = data.loc[:, 'label']


from collections import Counter

print(Counter(y))

Counter({0: 155, 1: 71})


In [0]:
id = X.loc[:,'id']
X = X.drop(columns=['id'])

In [0]:
from sklearn.model_selection import train_test_split
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.2)


X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid,
                                                       stratify=y_train_valid,
                                                       test_size=0.25)

print(len(y_test))
len(X_train)

46


135

In [0]:
train_path = "drive/datasets/smells_train.csv"
valid_path = "drive/datasets/smells_valid.csv"
test_path = "drive/datasets/smells_test.csv"

with open(train_path, "w") as file:  
  file.write(X_train.to_csv())
  
with open(valid_path, "w") as file:  
  file.write(X_valid.to_csv())
  
with open(test_path, "w") as file:  
  file.write(X_test.to_csv())

In [0]:
from torch.utils.data import Dataset, DataLoader

class SmellsDataset(Dataset):

    def __init__(self, data_path):

        self.dataset = pd.read_csv(data_path)
        
        rndperm = np.random.permutation(self.dataset.shape[0])

        self.dataset = self.dataset.iloc[rndperm[:],:].copy()
        
        self.dataset = self.dataset.iloc[:, 1:]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset.iloc[idx:idx+1,:]

        data = item.iloc[:,:-1]
        target = item.iloc[:,-1]
        
        return torch.tensor(data.values.astype(np.float32)), torch.tensor(target.values.astype(np.float32))

In [0]:
batch_size = 32
train_data = SmellsDataset(train_path)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=1, shuffle=True)


test_data = SmellsDataset(test_path)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
    num_workers=1)

valid_data = SmellsDataset(valid_path)

valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size,
                                          num_workers=1, shuffle=True)

In [0]:
batch, target = next(iter(train_loader))
batch.shape


torch.Size([32, 1, 896])

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(512, 341)
        self.dropout = nn.Dropout(0.4)
        self.fc3 = nn.Linear(341, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        x = x.view(-1, 512)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        return x

model = Net()
print(model)

In [0]:
import torch.optim


def train(dropout, best_valid_loss):
  model = Net()
  model.dropout = nn.Dropout(dropout)

  criterion = nn.BCELoss()

  optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)

  n_epochs = 100
  num_classes = 2

  model.train()

  counter = 0
  train_losses = []
  valid_losses = []
  
  for epoch in range(n_epochs):

      train_loss = 0.0

      for data, target in train_loader:
          counter += 1
          optimizer.zero_grad()
          
          output = model(data)

          loss = criterion(output, target.view(data.size(0)))

          loss.backward()

          optimizer.step()

          train_loss += loss.item()*data.size(0)

      val_losses = []
      model.eval()
      class_correct = list(0. for i in range(num_classes))
      class_total = list(0. for i in range(num_classes))
      for data, target in valid_loader:

          output = model(data)
          target = target.view(-1)

          loss = criterion(output, target)

          output = torch.round(output)

          correct = np.squeeze(output.eq(target.data.view_as(output)))
          for i in range(data.size(0)):
              label = int(target.data[i].item())

              class_correct[label] += correct[i].item()
              class_total[label] += 1

          val_losses.append(loss.item())

      f = 0
      if num_classes == 2:

            recall = class_correct[1] / class_total[1] if class_total[1] > 0 else 0
            precision = 0 if (class_correct[1] + (class_total[0] - class_correct[0])) == 0 else class_correct[1] / (class_correct[1] + (class_total[0] - class_correct[0]))
            f = 0 if precision + recall == 0 else (2 * precision * recall) / (precision + recall)

      model.train()

      train_loss = train_loss/len(train_loader.dataset)
      valid_loss = np.mean(val_losses)
      

      if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model, 'checkpoint.pth')
        print(best_valid_loss)
        print(f)
        print('..............................saving best')
      
      if epoch % 30 == 0:
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)

        
        print("Epoch: {}/{}...".format(epoch+1, n_epochs),
              "Loss: {:.4f}...".format(train_loss),
              "Val Loss: {:.4f}".format(valid_loss))

  return best_valid_loss

In [0]:
def tune_hyperparameters():

  best_valid_loss = 10000
  for dropout in [0.9,0.8,0.7,0.6]:
      for i in range(2):
        best_valid_loss = train(dropout, best_valid_loss)
  return best_valid_loss
      

In [0]:
best_valid_loss = tune_hyperparameters()

print('-------------------------------------------------')
model = torch.load('checkpoint.pth')
print(model)
print(best_valid_loss)

In [0]:
eps = 0.000001

orig_data = pd.read_csv("drive/datasets/features_lstm.csv")
orig_data = orig_data[orig_data['smell'] == 'feature-envy']

num_classes = 2

model = torch.load('checkpoint.pth')
model.to(torch.device('cpu'))

test_loss = 0.0
class_correct = list(0. for i in range(num_classes))
class_total = list(0. for i in range(num_classes))

model.eval()

for data, target in test_loader:

    output = model(data)
    target = target.view(-1)
    
    criterion = nn.BCELoss()
    
    loss = criterion(output, target)

    test_loss += loss.item()*data.size(0)

    output = torch.round(output)

    correct = np.squeeze(output.eq(target.data.view_as(output)))
   
    for i in range(data.size(0)):
      label = int(target.data[i].item())
      
      class_correct[label] += correct[i].item()
      class_total[label] += 1

test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(num_classes):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            class_correct[i],class_total[i]))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % i)

if num_classes == 2:
  recall = class_correct[1] / class_total[1] if class_total[1] > 0 else 0
  precision = class_correct[1] / (class_correct[1] + (class_total[0] - class_correct[0]))
  f = (2 * precision * recall) / (precision + recall)

  print("Recall: {:2f}...".format(recall),
  "Precision: {:2f}...".format(precision),
  "F-score: {:.2f}".format(f))
        
print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))